<a href="https://colab.research.google.com/github/apskis/data_science_ai_colab_books/blob/main/Session7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 7 — Interactive EDA with Plotly

Objectives
- Build interactive, exploratory plots with Plotly (plotly.express and plotly.graph_objects).
- Use Plotly FigureWidget for interactive notebook exploration.



In [ ]:
# Run in Colab
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

sns.set_style("whitegrid")
tips = sns.load_dataset("tips")
tips.head()


In [ ]:
# 1) Interactive histogram + KDE-like overlay (density via histogram_norm)
fig = px.histogram(tips, x="total_bill", nbins=30, histnorm="probability density",
                   marginal="rug", title="Total bill distribution (density)")
fig.update_traces(marker_color="#1f77b4", opacity=0.75)
fig.show()


In [ ]:
# 2) Interactive scatter with regression line (Plotly doesn't add regression by default; compute fit)
x = tips["total_bill"]
y = tips["tip"]
coef = np.polyfit(x, y, 1)
fit_y = np.poly1d(coef)(x)
fig = px.scatter(tips, x="total_bill", y="tip", color="day", hover_data=["size"],
                 title="Tip vs Total bill (interactive)")
fig.add_traces(go.Scatter(x=x, y=fit_y, mode="lines", line=dict(color="black", width=2),
                          name="Linear fit", hoverinfo="skip"))
fig.show()


In [ ]:
# 3) Box + jitter (violin-like interactivity): use facet or combined subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=("Boxplot by day", "Strip (jitter) by day"))
# Box
for i, d in enumerate(sorted(tips["day"].unique())):
    fig.add_trace(go.Box(y=tips.loc[tips.day==d, "tip"], name=d, boxmean="sd"), row=1, col=1)
# Jitter (strip)
fig.add_trace(go.Scatter(x=tips["day"], y=tips["tip"], mode="markers",
                         marker=dict(opacity=0.6, size=6, color="rgba(31,119,180,0.6)"),
                         name="points"), row=1, col=2)
fig.update_layout(height=450, showlegend=False, title_text="Box and points: tip by day")
fig.show()


In [ ]:
# 4) Interactive correlation heatmap (hover shows values)
num_cols = ["total_bill","tip","size"]
corr = tips[num_cols].corr()
fig = px.imshow(corr, text_auto=".2f", color_continuous_scale="RdBu", origin="lower",
                title="Correlation matrix (interactive)")
fig.update_layout(height=450)
fig.show()


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
day_of_week = ['Thur', 'Fri', 'Sat', 'Sun']

In [ ]:
# 5) Small interactive dashboard inside notebook using widgets (FigureWidget)
from ipywidgets import HBox, VBox, Dropdown, IntSlider, Output
from plotly.graph_objs import FigureWidget

fw = FigureWidget(px.scatter(tips, x="total_bill", y="tip", color="day"))
out = Output()

day_dropdown = Dropdown(options=["All"] + day_of_week, description="Day:")
size_slider = IntSlider(value=1, min=1, max=int(tips["size"].max()), description="Min size:")

def update_plot(change):
    sel_day = day_dropdown.value
    min_size = size_slider.value
    df = tips.copy()
    if sel_day != "All":
        df = df[df["day"] == sel_day]
    df = df[df["size"] >= min_size]
    with fw.batch_update():
        fw.data = []  # clear
        # re-add traces per day
        for d in sorted(df["day"].unique()):
            sub = df[df["day"]==d]
            fw.add_scatter(x=sub["total_bill"], y=sub["tip"], mode="markers", name=d)
    out.clear_output()
    with out:
        print(f"Showing {len(df)} rows — Day: {sel_day}, Min size: {min_size}")

day_dropdown.observe(update_plot, names="value")
size_slider.observe(update_plot, names="value")

display(VBox([HBox([day_dropdown, size_slider]), fw, out]))


In [ ]:
tips[(tips['day'] == 'Thur') & (tips['size'] >= 3)]